# Viscous Flow past a Circular Cylinder
This notebook aims to solve the Navier Stokes equations past a circular cylinder. We use some data from a lower order potential flow solution for flow past a circular cylinder and see whether a PINN can predict the boundary layer and separation in the flow.

In [81]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

### Physics-infromed Neural Network Class

In [82]:
class PINN(tf.keras.Model):
    def __init__(self, layers):
        super(PINN, self).__init__()
        # Initialize hidden layers and output layer with custom weight initialization
        self.hidden_layers = [tf.keras.layers.Dense(layer, activation='tanh') for layer in layers[:-1]]
        self.output_layer = tf.keras.layers.Dense(layers[-1])

    def build(self, input_shape):
        # Manually build each layer to ensure proper initialization
        for layer in self.hidden_layers:
            layer.build(input_shape)
            input_shape = (input_shape[0], layer.units)
        self.output_layer.build(input_shape)
        super(PINN, self).build(input_shape)

    def call(self, inputs):
        # Forward pass through the network
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)

### Declaring an instance of the PINN class

In [83]:

layers = [2, 70, 50, 30, 10, 3]   
# Outputs: U, V, P, 

model = PINN(layers)
# declare the model

#hyperparameters
#learning_rate (float or schedule, default=0.001): The step size for updating weights during optimization.
#beta_1 (float, default=0.9): Exponential decay rate for the first moment estimates (mean of gradients).
#beta_2 (float, default=0.999): Exponential decay rate for the second moment estimates (variance of gradients).
#amsgrad (boolean, default=False):Whether to apply the AMSGrad variant of the optimizer.
lr = 0.001
b1 = 0.9
b2 = 0.999
optimizer = tf.keras.optimizers.Adam(learning_rate=lr,beta_1=b1,beta_2=b2,amsgrad=False)

In [84]:
model.build(input_shape=(None, 2)) 
model.summary()

Model: "pinn_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 70)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 50)             │         3,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 30)             │         1,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,639 (22.03 KB)

 Trainable params: 5,639 (22.03 KB)

 Non-trainable params: 0 (0.00 B)

## Governing Equations and Loss
The continuity eqaution
$$
\frac{\partial U}{\partial x} + \frac{\partial V}{\partial y} = 0
$$

In [85]:
def continuity_loss(model, x, y):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, y])
        inputs = tf.stack([x, y], axis=1)[:,:,0]
        predictions = model(inputs)
        U = predictions[:, 0]  
        V = predictions[:, 1]  
    dU_dx = tape.gradient(U, x)
    dV_dy = tape.gradient(V, y)
    
    continuity_residual = dU_dx + dV_dy
    continuity_loss = tf.reduce_mean(tf.square(continuity_residual))

    return continuity_loss

The X-momentum equation
$$
U \frac{\partial U}{\partial x} + V \frac{\partial U}{\partial y} = -\frac{\partial P}{\partial x} 
+ \frac{1}{\mathrm{Re}} \left( \frac{\partial^2 U}{\partial x^2} + \frac{\partial^2 U}{\partial y^2} \right) 
$$

In [86]:
def x_momentum_loss(model, x, y, Re):
    with tf.GradientTape(persistent=True) as tape2:
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch([x, y])
            tape2.watch([x, y])
            inputs = tf.stack([x, y], axis=1)[:,:,0]
            predictions = model(inputs)
            U = predictions[:, 0]  # U: x-velocity
            V = predictions[:, 1]  # V: y-velocity
            P = predictions[:, 2]  # P: pressure
        
        # First-order derivatives
        dU_dx = tape1.gradient(U, x)
        dU_dy = tape1.gradient(U, y)
        dP_dx = tape1.gradient(P, x)
        
        # Second-order derivatives
        d2U_dx2 = tape2.gradient(dU_dx, x)
        d2U_dy2 = tape2.gradient(dU_dy, y)

    # X-momentum equation residual
    x_momentum_residual = U*dU_dx + V*dU_dy + dP_dx -(1/Re)*(d2U_dx2+d2U_dy2)
    x_momentum_loss = tf.reduce_mean(tf.square(x_momentum_residual))

    return x_momentum_loss

The Y-momentum equation
$$
U \frac{\partial V}{\partial x} + V \frac{\partial V}{\partial y} = 
-\frac{\partial P}{\partial y} 
+ \frac{1}{\mathrm{Re}} \left( \frac{\partial^2 V}{\partial x^2} + \frac{\partial^2 V}{\partial y^2} \right) 
$$

In [87]:
def y_momentum_loss(model, x, y, Re):
    with tf.GradientTape(persistent=True) as tape2:
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch([x, y])
            tape2.watch([x, y])
            inputs = tf.stack([x, y], axis=1)[:,:,0]
            predictions = model(inputs)
            U = predictions[:, 0]  # U: x-velocity
            V = predictions[:, 1]  # V: y-velocity
            P = predictions[:, 2]  # P: pressure

        
        # First-order derivatives
        dV_dx = tape1.gradient(V, x)
        dV_dy = tape1.gradient(V, y)
        dP_dy = tape1.gradient(P, y)

        
        # Second-order derivatives
        d2V_dx2 = tape2.gradient(dV_dx, x)
        d2V_dy2 = tape2.gradient(dV_dy, y)

    # X-momentum equation residual
    y_momentum_residual = U*dV_dx + V*dV_dy + dP_dy -(1/Re)*(d2V_dx2+d2V_dy2)
    y_momentum_loss = tf.reduce_mean(tf.square(y_momentum_residual))

    return y_momentum_loss

## Boundary Conditions

No-slip \& No-penetration at the cylindrical boundary
$$
U = V = 0
$$

In [88]:
def wall_boundary_loss(model, x_wall, y_wall):
    inputs = tf.stack([x_wall, y_wall], axis=1)[:,:,0]
    predictions = model(inputs)
    U = predictions[:, 0]  # U: x-velocity
    V = predictions[:, 1]  # V: y-velocity


    velocity_residual = tf.square(U) + tf.square(V)
    # Combine losses
    wall_loss = tf.reduce_mean(velocity_residual)
    return wall_loss

Inflow 
$$
U = 1
$$
$$
V = 0
$$

In [89]:
def inflow_boundary_loss(model, x_in, y_in):

    inputs = tf.stack([x_in, y_in], axis=1)[:,:,0]
    predictions = model(inputs)
    U = predictions[:, 0]  # U: x-velocity
    V = predictions[:, 1]  # V: y-velocity

    # Residuals for boundary conditions
    velocity_residual = tf.square(U - 1) + tf.square(V)  


    # Combine losses
    inflow_loss = tf.reduce_mean(velocity_residual)

    return inflow_loss

Upper and lower free boundaries

$$
\frac{\partial U}{\partial y} = 0
$$
$$
V = 0
$$

In [90]:
def upper_lower_boundary_loss(model, x_combined, y_combined):


    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x_combined, y_combined])
        # Stack inputs for the model
        inputs = tf.stack([x_combined, y_combined], axis=1)[:,:,0]
        predictions = model(inputs)
        
        # Extract predicted values
        U  = predictions[:, 0]  # U: x-velocity
        V  = predictions[:, 1]  # V: y-velocity


    # Compute derivatives with respect to y
    dU_dy = tape.gradient(U, y_combined)


    # Residuals for boundary conditions
    velocity_residual = tf.square(V) 
    derivative_residual = tf.square(dU_dy) 

    # Combine losses
    upper_lower_loss = tf.reduce_mean(velocity_residual + derivative_residual)
    return upper_lower_loss

Outflow 
$$
\frac{2}{Re}\frac{\partial U}{\partial x} - p = 0
$$
$$
\frac{2}{Re}\frac{\partial V}{\partial y} - p = 0
$$
$$
 \frac{\partial U}{\partial y} + \frac{\partial V}{\partial x} =  0
$$


In [91]:
def outflow_loss(model, x_out, y_out,Re):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x_out, y_out])
        # Stack inputs for the model
        inputs = tf.stack([x_out, y_out], axis=1)[:,:,0]
        predictions = model(inputs)
        
        # Extract predicted values
        U = predictions[:, 0]  # U: x-velocity
        V = predictions[:, 1]  # V: y-velocity
        P = predictions[:, 2]  # P: pressure


    # Compute derivatives with respect to x
    dU_dx = tape.gradient(U, x_out)
    dV_dx = tape.gradient(V, x_out)
    dU_dy = tape.gradient(U, y_out)
    dV_dy = tape.gradient(V, y_out)


    # Residuals for zero-gradient conditions
    residual =   tf.square((2/Re)*dU_dx - P) + tf.square((2/Re)*dV_dy - P) + tf.square(dU_dy+ dV_dx)
    
    # Combine losses
    outflow_loss = tf.reduce_mean(residual)

    return outflow_loss

### Data Loss: Using Potential Flow Solution for Training the RANS PINN

Earlier, a PINN model has been trained to accurately predict the inviscid, icompressible flow past a cylinder. The idea is to use that model for training data in the upstream of the cylinder.

<img src="potential flow solution.png" alt="Description of image" style="width: 800px;"/>

Note that only the part upstream of the cylinder and away from the walls will be used since this is the region where the invisicid assumptions are acceptable.

In [92]:
import data_loss

In [93]:
import Points

<img src="Training_points_1.jpg" alt="Description of image" style="width: 900px;"/>

In [94]:
data_x = data_loss.X
data_y = data_loss.Y
data_U = data_loss.U
data_V = data_loss.V
data_P = data_loss.P

In [95]:
def data_pot_loss(model,x,y,u,v,p):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, y])
        # Stack inputs for the model
        inputs = tf.stack([x, y], axis=1)[:,:,0]
        predictions = model(inputs)
        
        # Extract predicted values
        U = predictions[:, 0]  # U: x-velocity
        V = predictions[:, 1]  # V: y-velocity
        P = predictions[:, 2]
    loss = tf.square(U-u) + tf.square(V-v) + tf.square(P-p)
    return tf.reduce_mean(loss)

In [96]:
xint = Points.xinterior
yint = Points.yinterior

In [97]:
xwall = Points.x_wall
ywall = Points.y_wall

In [98]:
xin = Points.xinflow
yin = Points.yinflow
xout = Points.xoutflow
yout = Points.youtflow
xfar = Points.xfar
yfar = Points.yfar

In [99]:

totalloss = []

inflow = []
outflow = []
farflow = []
wallloss = []

cont = []
xmom = []
ymom = []

dataloss = []
epochnum = []
Re = 10

Ny = 1000
Nx = 2000
xgrid = np.linspace(-Points.a,Points.b,Nx)
ygrid = np.linspace(-Points.a,Points.a,Ny)
xmesh,ymesh = np.meshgrid(xgrid,ygrid)
x_flat = tf.convert_to_tensor(xmesh.flatten().reshape(-1, 1), dtype=tf.float32)
y_flat = tf.convert_to_tensor(ymesh.flatten().reshape(-1, 1), dtype=tf.float32)
xy_tensor = tf.concat([x_flat, y_flat], axis=1)
mask = (xmesh**2 + ymesh**2 < 1)
predictions = model(xy_tensor)



In [ ]:
n_epochs = 10000
for epoch in range(n_epochs):
    with tf.GradientTape() as tape:
        
        if(epoch<1000):
            pdeweight = 1
            bcweight1 = 10
            bcweight2 = 0.1
            data_weight = 10
     
            
        epochnum.append(epoch)
        cont.append(    pdeweight*tf.cast(continuity_loss(model,xint,yint   ),dtype = tf.float32))
        xmom.append(    pdeweight*tf.cast(x_momentum_loss(model,xint,yint,Re),dtype = tf.float32))
        ymom.append(    pdeweight*tf.cast(y_momentum_loss(model,xint,yint,Re),dtype = tf.float32))
        
        inflow.append(  bcweight1*tf.cast(inflow_boundary_loss(model,xin,yin),dtype = tf.float32))
        outflow.append( bcweight1*tf.cast(outflow_loss(       model,xout,yout,Re             ),dtype = tf.float32))
        farflow.append( bcweight2*tf.cast(upper_lower_boundary_loss(model,xfar,yfar),dtype = tf.float32))
        wallloss.append(bcweight2*tf.cast(wall_boundary_loss(model,xwall,ywall),dtype = tf.float32))
        dataloss.append(data_weight*tf.cast(data_pot_loss(model,data_x,data_y,data_U,data_V,data_P),dtype = tf.float32))
            
        
        loss = ((cont[-1] + xmom[-1] + ymom[-1]) + (inflow[-1] + outflow[-1] + farflow[-1] + wallloss[-1]) + dataloss[-1])#/#(3*pdeweight+2*bcweight2+2*bcweight1+data_weight)

        totalloss.append(loss)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        print("n = ", epoch,", loss = ",loss.numpy())
        if(epoch% 20==0):
            #print("n = ", epoch,", loss = ",loss.numpy())  
            #clear_output(wait=True)
            
            plt.figure(figsize=(10,5))
            plt.semilogy(totalloss,label = "total loss")
            plt.semilogy(cont,label = "continuity")
            plt.semilogy(xmom,label = "x momentum")
            plt.semilogy(ymom,label = "y momentum")
            plt.semilogy(inflow,label = "inflow bc")
            plt.semilogy(outflow,label = "outflow bc")
            plt.semilogy(farflow,label = "farfield bc")
            plt.semilogy(wallloss,label = "wall bc")
            plt.semilogy(dataloss,label = "Potential Flow")
            plt.legend()
            plt.title("MSE (with data)")
            plt.grid()
            plt.savefig("MSE with data.jpg")
            plt.close()
        if (epoch%100 == 0):
           # model.export('./Model')
            predictions = model(xy_tensor)
            
            # Extract individual components from predictions

            U  = predictions[:, 0].numpy().reshape(xmesh.shape)
            V  = predictions[:, 1].numpy().reshape(xmesh.shape)
            P  = predictions[:, 2].numpy().reshape(xmesh.shape)

            U[mask]  = np.nan
            V[mask]  = np.nan
            P[mask]  = np.nan
            fields = {'U': U, 'V': V, 'P': P, }
            plt.figure(figsize=(15, 10))
            for i, (field_name, field_data) in enumerate(fields.items(), 1):
                plt.subplot(2, 3, i)
                contour = plt.contourf(xmesh, ymesh, field_data, levels=50, cmap='viridis')
                plt.colorbar(contour, label=field_name)
                plt.title(f"{field_name} Contour")
                plt.xlabel("x")
                plt.ylabel("y")
                plt.gca().set_aspect('equal')



            plt.tight_layout()
            plt.savefig("Plot" +str(epoch) +".png")
            plt.close()


    

n =  0 , loss =  22.771713
n =  1 , loss =  16.612137
n =  2 , loss =  11.899753
n =  3 , loss =  8.553914
n =  4 , loss =  6.377908
n =  5 , loss =  5.101742
n =  6 , loss =  4.449416
n =  7 , loss =  4.187975
n =  8 , loss =  4.1459017
n =  9 , loss =  4.206899
n =  10 , loss =  4.294173
n =  11 , loss =  4.3572884
n =  12 , loss =  4.3651643
n =  13 , loss =  4.3041725
n =  14 , loss =  4.177534
n =  15 , loss =  4.001445
n =  16 , loss =  3.798286
n =  17 , loss =  3.589999
n =  18 , loss =  3.3934798
n =  19 , loss =  3.2186298
n =  20 , loss =  3.0688496
n =  21 , loss =  2.9428608
n =  22 , loss =  2.8365436
n =  23 , loss =  2.7441378
n =  24 , loss =  2.659157
n =  25 , loss =  2.5755863
n =  26 , loss =  2.4891915
n =  27 , loss =  2.398103
n =  28 , loss =  2.3024132
n =  29 , loss =  2.2034128
n =  30 , loss =  2.1031601
n =  31 , loss =  2.0044951
n =  32 , loss =  1.9110808
n =  33 , loss =  1.826977
n =  34 , loss =  1.7556255
n =  35 , loss =  1.6986765
n =  36 , loss =

In [21]:

# Plotting results

plt.figure(figsize=(15, 10))
for i, (field_name, field_data) in enumerate(fields.items(), 1):
    plt.subplot(2, 3, i)
    contour = plt.contourf(xmesh, ymesh, field_data, levels=50, cmap='viridis')
    plt.colorbar(contour, label=field_name)
    plt.title(f"{field_name} Contour")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.gca().set_aspect('equal')
    
    

plt.tight_layout()
plt.savefig("Plot.png")
plt.close()